# Fully connected NN with tensorflow

First we import the necessary modules

In [1]:
import tensorflow as tf
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from data_handler_CMAPS import CMAPSDataHandler
#import tensorflow.contrib.layers.xavier_initializer as xavier_initializer

# Define the NN architecture

In [2]:
def create_placeholders(input_shape, output_shape):
    
    X = tf.placeholder(tf.float32, shape=(None,input_shape), name="X")
    y = tf.placeholder(tf.float32, shape=None, name="y")
    
    return X, y

def ann_initialization(input_shape, output_shape):


    tf.reset_default_graph()
    
    X, y = create_placeholders(input_shape, output_shape)
    
    weights = {}
    biases = {}
    
    #Weights declaration
        
    W1 = tf.get_variable(name="W1", shape=(input_shape, 20), initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    b1 =  tf.get_variable(name="b1", shape=(1, 20), initializer=tf.zeros_initializer())
    weights["W1"] = W1
    biases["b1"] = b1
    
    W2 = tf.get_variable(name="W2", shape=(20, 20), initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    b2 =  tf.get_variable(name="b2", shape=(1, 20), initializer=tf.zeros_initializer())
    weights["W2"] = W2
    biases["b2"] = b2
    
    W3 = tf.get_variable(name="W3", shape=(20, output_shape), initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    b3 =  tf.get_variable(name="b3", shape=(1, output_shape), initializer=tf.zeros_initializer())
    weights["W3"] = W3
    biases["b3"] = b3
    
        
    return {"X":X, "y":y, "W":weights, "b":biases}



# Function to train the neural network

In [8]:
def multilayer_perceptron(X, weights, biases):
    
    #print(weights["W1"].shape)
    #print(X.shape)
    
    W1, b1 = weights["W1"], biases["b1"]
    W2, b2 = weights["W2"], biases["b2"]
    W3, b3 = weights["W3"], biases["b3"]
    
    Z1 = tf.add(tf.matmul(X, W1), b1)
    A1 = tf.nn.relu(Z1)
    
    Z2 = tf.add(tf.matmul(A1, W2), b2)
    A2 = tf.nn.relu(Z2)
    
    out = tf.add(tf.matmul(A2, W3), b3)
    
    return out 


def multilayer_perceptron2(X):
    
    #print(weights["W1"].shape)
    #print(X.shape)
    
    l2_regularizer = 0.0
    
    X = tf.layers.dense(X, 20, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
                    kernel_regularizer=tf.contrib.layers.l2_regularizer(l2_regularizer))
    
    X = tf.layers.dense(X, 20, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
                    kernel_regularizer=tf.contrib.layers.l2_regularizer(l2_regularizer))
    
    X = tf.layers.dense(X, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    
    return X 


def train_ann(X_train, y_train, model, epochs, batch_size, display_step = 1, lamba2 = 0.0):
    
    input_shape = X_train.shape[1]
    output_shape = 1
    
    #model = ann_initialization(input_shape, output_shape)
    X, y = model["X"], model["y"]
    #X = tf.placeholder(tf.float32, shape=None, name="X")
    #y = tf.placeholder(tf.float32, shape=None, name="y")
    #print(X)
    #print(y)
    #print(weights)
    #print(biases)
    
    #predictions = multilayer_perceptron(X, weights, biases)
    predictions = multilayer_perceptron2(X)
    #regularizers = lamba2*(tf.nn.l2_loss(weights["W1"]) + tf.nn.l2_loss(weights["W2"]))
    #cost = tf.losses.mean_squared_error(y, predictions) + regularizers
    cost = tf.losses.mean_squared_error(y, predictions)
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost)
    
    with tf.Session() as sess:
        
        sess.run(tf.global_variables_initializer())
        avg_cost = 0.0
        
        for epoch in range(epochs):
            
            cost_tot = 0.0
            total_batch = int(X_train.shape[0]/batch_size)
            X_batches = np.array_split(X_train, total_batch)
            #print(len(X_batches))
            y_batches = np.array_split(y_train, total_batch)
            #print(len(y_batches))
            
            #Train with the minibatches
            for i in range(total_batch):
                batch_x, batch_y = X_batches[i], y_batches[i]
                
                #print(batch_x.shape)
                #print(batch_y.shape)
                _, c = sess.run([optimizer, cost], feed_dict={X:batch_x, y:batch_y})
                cost_tot += c
                
            avg_cost = cost_tot/total_batch
                
            if epoch%display_step == 0:
                print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
                    
        print("Training complete!")
        print("Epoch:Final", "cost=", "{:.9f}".format(avg_cost))
        
        return (X, y, weights, biases)  

# Create the Data Handler

In [4]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 128

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul=max_rul,
                                  sequence_length=window_size, sequence_stride=window_stride, 
                                  data_scaler=min_max_scaler)
dHandler_cmaps.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
dHandler_cmaps.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 128. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(17731, 420)
(17731, 1)
Testing data (X, y)
(100, 420)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.63253012 -0.18639634 -0.38048616 ..., -0.66666667  0.41085271
   0.42723005]
 [-0.43373494 -0.09396119 -0.29473329 ..., -0.33333333  0.24031008
   0.21817178]
 [-0.31325301 -0.26095487 -0.25894666 ..., -0.33333333  0.36434109
   0.67274234]
 [-0.31325301 -0.48768258 -0.33760972 ..., -0.33333333  0.06976744
   0.2604253 ]
 [-0.30120482 -0.48506649 -0.19074949 ..., -0.33333333  0.03875969
   0.14609224]]
[[ 128.]
 [ 128.]
 [ 128.]
 [ 128.]
 [ 128.]]
Testing data (X, y)
[[-0.69879518 -0.24089819 -0.55536799 ..., -0.16666667  0.03875969
   0.27312897]
 [-0.09036145 -0.21037715 -0.37744767 ..., -0.5         0.03875969
   0.01518917]
 [-0.0060241  -0.22127752 -0.10634706 ...,  0.16666667 

# Train the ANN

In [9]:
window_size = 30
window_stride = 1
max_rul = 128
num_features = len(selected_features)
shape = num_features*window_size


#Create the model
model = ann_initialization(shape, 1)
#print(model)

#Train the model
X = dHandler_cmaps.X_train
y = np.ravel(dHandler_cmaps.y_train)

epochs = 100
batch_size = 512

#print(X.shape)
#print(y.shape)
model = train_ann(X, y, model, epochs=epochs, batch_size=batch_size, display_step = 1, lamba2 = 0)

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
Epoch: 0001 cost= 7768.857881434
Epoch: 0002 cost= 5045.210039924
Epoch: 0003 cost= 2968.148100011
Epoch: 0004 cost= 2304.186911190
Epoch: 0005 cost= 1946.001375086
Epoch: 0006 cost= 1892.682750029
Epoch: 0007 cost= 1885.108121984
Epoch: 0008 cost= 1878.437808766
Epoch: 0009 cost= 1873.182211483
Epoch: 0010 cost= 1869.061598834
Epoch: 0011 cost= 1865.699667538
Epoch: 0012 cost= 1863.000337489
Epoch: 0013 cost= 1860.830146341
Epoch: 0014 cost= 1859.050860237
Epoch: 0015 cost= 1857.652709961
Epoch: 0016 cost= 1856.519628188
Epoch: 0017 cost= 1855.601645077
Epoch: 0018 cost= 1854.878690832
Epoch: 0019 cost= 1854.285184972
Epoch: 0020 cost= 1853.808101878
Epoch: 0021 cost= 1853.426901425
Epoch: 0022 cost= 1853.116519704
Epoch: 0023 cost= 1852.856028837
Epoch: 0024 cost= 1852.615431842
Epoch: 0025 cost= 1852.422736673
Epoch: 0026 cost= 1852.256771312
Epoch: 0027 cost= 1852.113992130
Epoch: 0028

NameError: name 'weights' is not defined